<a href="https://colab.research.google.com/github/Serena-Wang/Anagrams/blob/master/season_week_kernel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/tensorflow/probability.git

Cloning into 'probability'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 31309 (delta 30), reused 26 (delta 1), pack-reused 31251
Receiving objects: 100% (31309/31309), 47.84 MiB | 28.60 MiB/s, done.
Resolving deltas: 100% (26140/26140), done.


In [0]:
import sys
sys.path.append('/content/probability')

In [3]:
cd /content/probability/tensorflow_probability

/content/probability/tensorflow_probability


In [0]:
#!pip install --upgrade tensorflow

In [0]:
#import probability

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

%tensorflow_version 2.x
#import tensorflow as tf
import tensorflow.compat.v2 as tf

from tensorflow_probability.python.internal import assert_util
from tensorflow_probability.python.internal import tensor_util
from tensorflow_probability.python.math.psd_kernels.internal import util
from tensorflow_probability.python.math.psd_kernels import exponentiated_quadratic
from tensorflow_probability.python.math.psd_kernels.positive_semidefinite_kernel import PositiveSemidefiniteKernel


TensorFlow 2.x selected.


In [0]:
__all__ = ['season_week_kernel']

In [0]:
class season_week_kernel(PositiveSemidefiniteKernel):
  """The season_week_kernel kernel.
  Sometimes called the "squared exponential", "Gaussian" or "radial basis
  function", this kernel function has the form
    ```none
    k(x, y) = amplitude**2 * exp(-||x - y||**2 / (2 * length_scale**2))
    ```
  where the double-bars represent vector length (ie, Euclidean, or L2 norm).
  """

  def __init__(self,
               amplitude=None,
               length_scale=None,
               feature_ndims=1,
               validate_args=False,
               name='season_week_kernel'):
    """Construct an ExponentiatedQuadratic kernel instance.
    Args:
      amplitude: floating point `Tensor` that controls the maximum value
        of the kernel. Must be broadcastable with `length_scale` and inputs to
        `apply` and `matrix` methods. Must be greater than zero. A value of
        `None` is treated like 1.
        Default value: None
      length_scale: floating point `Tensor` that controls how sharp or wide the
        kernel shape is. This provides a characteristic "unit" of length against
        which `||x - y||` can be compared for scale. Must be broadcastable with
        `amplitude` and inputs to `apply` and `matrix` methods. A value of
        `None` is treated like 1.
        Default value: None
      feature_ndims: Python `int` number of rightmost dims to include in the
        squared difference norm in the exponential.
      validate_args: If `True`, parameters are checked for validity despite
        possibly degrading runtime performance
      name: Python `str` name prefixed to Ops created by this class.
    """
    parameters = dict(locals())
    with tf.name_scope(name):
      dtype = util.maybe_get_common_dtype(
          [amplitude, length_scale])
      self._amplitude = tensor_util.convert_nonref_to_tensor(
          amplitude, name='amplitude', dtype=dtype)
      self._length_scale = tensor_util.convert_nonref_to_tensor(
          length_scale, name='length_scale', dtype=dtype)
      
      super(ExponentiatedQuadratic, self).__init__(
          feature_ndims,
          dtype=dtype,
          name=name,
          validate_args=validate_args,
          parameters=parameters)

  @property
  def amplitude(self):
    """Amplitude parameter."""
    return self._amplitude

  @property
  def length_scale(self):
    """Length scale parameter."""
    return self.self._length_scale

  ####????
  ##This property describes the fully broadcast shape of all kernel parameters
  def _batch_shape(self):
    scalar_shape = tf.TensorShape([])
    return tf.broadcast_static_shape(
        scalar_shape if self.amplitude is None else self.amplitude.shape,
        scalar_shape if self.length_scale is None else self.length_scale.shape)
  ####????
  def _batch_shape_tensor(self):
    return tf.broadcast_dynamic_shape(
        [] if self.amplitude is None else tf.shape(self.amplitude),
        [] if self.length_scale is None else tf.shape(self.length_scale))

  def _apply(self, x1, x2, example_ndims=0):
    # x1, x2 shoule be #obs x 2
    season1 = x1[:,0:]
    week1 = x1[:,1:] 
    season2 = x2[:,0:]
    week2 = x2[:,1:]

    k = tfk.ExponentiatedQuadratic(self.amplitude[0], self.length_scale[0])
    k._apply(week1,week2)

    same_season = (season1 == season2)
    ks = tfk.ExponentiatedQuadratic(self.amplitude[1], self.length_scale[1])
    ks._apply(week1[same_season, 1:],week2[same_season, 1:])

    if (season1 != season2):
      return k
    else:
      return k+ks

  def _parameter_control_dependencies(self, is_init):
    if not self.validate_args:
      return []
    assertions = []
    for arg_name, arg in dict(amplitude=self.amplitude,
                              length_scale=self.length_scale).items():
      if arg is not None and is_init != tensor_util.is_ref(arg):
        assertions.append(assert_util.assert_positive(
            arg,
            message='{} must be positive.'.format(arg_name)))
    return assertions

In [0]:
x = f.constant([5, 4, 6])

season1 = x1[,0]
week1 = x1[,1]
season2 = x2[,0]
week2 = x2[,1]



k = tfk.ExponentiatedQuadratic(self.length_scale[0], self.length_scale[0])
k._apply(week1,week2)

same_season = (season1 == season2)
ks = tfk.ExponentiatedQuadratic(self.length_scale[1], self.length_scale[1])
ks._apply(week1[same_season, :],week2[same_season, :])

In [0]:
import tensorflow_probability as tfp
tfd = tfp.distributions
import numpy as np
import pandas as pd
tfb = tfp.bijectors

In [26]:
amplitude_var = tfp.util.TransformedVariable(
    initial_value=1.,
    bijector=constrain_positive,
    name='amplitude',
    dtype=np.float64)

length_scale_var = tfp.util.TransformedVariable(
    initial_value=1.,
    bijector=constrain_positive,
    name='length_scale',
    dtype=np.float64)

NameError: ignored

In [16]:
amplitude

<tfp.distributions.LogNormal 'LogNormal' batch_shape=[] event_shape=[] dtype=float64>

In [23]:
# Suppose `SomeKernel` acts on vectors (rank-1 tensors)
test = season_week_kernel(amplitude,length_scale)
test.batch_shape
# ==> []


ValueError: ignored

In [0]:

# `x` and `y` are batches of five 3-D vectors:
x = np.ones([5, 3], np.float32)
y = np.ones([5, 3], np.float32)
scalar_kernel.apply(x, y).shape
# ==> [5]